## **PROBLEM 1**

**Problem Formulation:**
*   *Representation:*
For this problem, I chose to represent each individuals using a list which contains two integer values representing the x and y points. Both points are subject to the following upon initialization: −60 ≤ x ≤ 40, −30 ≤ y ≤ 70. To achieve this, a customInitRepeat() function was created as well as a createAttribute() function.

*   *Parenthood selection:*
For the parent selection, the program uses the varAnd() function which iterates over the offspring and produces a random number each iteration. If said number is less than the crossover probability, then individual[i] is mated with individual[i+1] which is the next individual in the list.

*   *Mutation:*
For mutation, the provided .mutUniformInt() mutation tool was used. It mutates an individual by replacing its attributes, the advantage of this is that it allows us to set an upper bound and lower bound, ensuring that the previous contrainsts for x and y are still followed.

*   *Crossover:*
for crossover, a simple one-point cross over was employed using the provided .cxOnePoint() mating tool. This interchanges the points between two parents.

*  *Survival selection:*
In order to select the survivors, 2-way tournament selection was used. The provided .selTournament() selection tool was utilized. The evalutation function contained the given objective function.

### Initializations and functions

In [29]:
import random, math, time
from deap import algorithms, base, creator, tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Evalutaion function using the objective function
# (|x| + |y|) · (1 + |sin(|x| · π)| + |sin(|y| · π)|) 
def eval(individual):
    x = individual[0]
    y = individual[1]
    ret = (abs(x) + abs(y)) * (1 + abs(math.sin(abs(x) * math.pi)) + abs(math.sin(abs(y) * math.pi)))
    return (ret,)

# Evalutaion function using the objective function
# (|x| + |y|) · (1 + |sin(3 · |x| · π)| + |sin(3 · |y| · π)|) 
def eval2(individual):
    x = individual[0]
    y = individual[1]
    ret = (abs(x) + abs(y)) * (1 + abs(math.sin(3 * abs(x) * math.pi)) + abs(math.sin(3 * abs(y) * math.pi)))
    return (ret,)

# Custom initRepeat that takes in n as parameter for the function being called
def customInitRepeat(container, func, n):
    return container(func(n) for n in range(n))

# Returns an integer x, wherein -60 <= x <= 40 if its the first elements and -30 <= x <= 70 if it is the second element
def createAttribute(n):
    return random.randint(-60, 40) if n == 0 else random.randint(-30, 70)

# hill-climb local search on each individual in population
def local_search(initPop):
    # operations on x and y: +_, _+, -_, _-
    pop = []

    for ind in initPop:
        x = ind[0]
        y = ind[1]
        pop.append(creator.Individual([x + 1, y]))
        pop.append(creator.Individual([x - 1, y]))
        pop.append(creator.Individual([x, y + 1]))
        pop.append(creator.Individual([x, y - 1]))
    
    return pop

toolbox = base.Toolbox()

IND_SIZE = 2 # x and y

toolbox.register("attr_bool", createAttribute)
toolbox.register("individual", customInitRepeat, creator.Individual, toolbox.attr_bool, IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low = [-60, -30], up = [30, 70], indpb=1)
toolbox.register("select", tools.selTournament, tournsize=2)

### Random initialization (with different random initial populations)

In [30]:
for i in range(10):
    popSize = 100
    pop = toolbox.population(n=popSize)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    print(f'* Population size: {popSize} => Best points: ({tools.selBest(pop, k=1)[0][0]}, {tools.selBest(pop, k=1)[0][1]})') 
    print()

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (-1, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -2)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (2, 2)



### Holding a 2-way tournament selection

In [31]:
for i in range(10):
    initPopSize = 200
    popSize = 100
    pop = toolbox.population(n=initPopSize)
    pop = toolbox.select(pop, popSize)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    print(f'* Population size: {popSize} => Best points: ({tools.selBest(pop, k=1)[0][0]}, {tools.selBest(pop, k=1)[0][1]})') 
    print()

* Population size: 100 => Best points: (-1, 1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (-1, 0)

* Population size: 100 => Best points: (-1, -1)

* Population size: 100 => Best points: (-1, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -1)

* Population size: 100 => Best points: (-1, -1)

* Population size: 100 => Best points: (1, 1)



### Multi-Start Local Search

initial population is 25, perform local serach on each individual by moving one unit. 4 directions means 4 offspring from local search. Populate a new list with this method then it should result in a population of 100. Use this as initial population for EA.

In [32]:
for i in range(10):
    initPopSize = 25
    pop = toolbox.population(n=initPopSize)
    pop = local_search(pop)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    print(f'* Population size: {len(pop)} => Best points: ({tools.selBest(pop, k=1)[0][0]}, {tools.selBest(pop, k=1)[0][1]})') 
    print()

* Population size: 100 => Best points: (0, -1)

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (1, 0)

* Population size: 100 => Best points: (2, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -1)

* Population size: 100 => Best points: (-1, -1)



## **PROBLEM 2**

In [33]:
# Uses the evaluation function for problem 2
toolbox.register("evaluate", eval2)

### Random initialization (with different random initial populations)

In [34]:
for i in range(10):
    popSize = 100
    pop = toolbox.population(n=popSize)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    print(f'* Population size: {popSize} => Best points: ({tools.selBest(pop, k=1)[0][0]}, {tools.selBest(pop, k=1)[0][1]})') 
    print()

* Population size: 100 => Best points: (0, 0)



* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (1, -1)

* Population size: 100 => Best points: (-1, -1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 1)



### Holding a 2-way tournament selection

In [35]:
for i in range(10):
    initPopSize = 200
    popSize = 100
    pop = toolbox.population(n=initPopSize)
    pop = toolbox.select(pop, popSize)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    print(f'* Population size: {popSize} => Best points: ({tools.selBest(pop, k=1)[0][0]}, {tools.selBest(pop, k=1)[0][1]})') 
    print()

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (2, 0)

* Population size: 100 => Best points: (0, -1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -1)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (1, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -1)



### Multi-Start Local Search

In [36]:
for i in range(10):
    initPopSize = 25
    pop = toolbox.population(n=initPopSize)
    pop = local_search(pop)
    result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    print(f'* Population size: {len(pop)} => Best points: ({tools.selBest(pop, k=1)[0][0]}, {tools.selBest(pop, k=1)[0][1]})') 
    print()

* Population size: 100 => Best points: (1, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 1)

* Population size: 100 => Best points: (-1, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -2)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, 0)

* Population size: 100 => Best points: (0, -1)



## **PROBLEM 3**

*Analysis on Runtime:* 
The increase in the number of local optima seems to have affected Multi-Start local. Increasing the number of local optima resulted in longer run times due to increased exploration costs, convergence challenges, larger search space sizes, algorithm sensitivity, and potential divergence in solution quality. 

*Analysis on fitness values:*
In terms of the fitness values of the 3 approaches, regardless of the number of local optima, there was no significant difference in the fitness values of the best individuals that resulted in all of the runs. Perhaps, lessening the number of generations might amplify the difference among the different approaches.

*Analysis on convergence rate:*
In terms of convergence rate, the 2-way tournament selection, when employed on the first problem (less local optima) seems to arrive at the solution the least amount of times. This is not the predicted behavior and is a glimpse to the stochastic nature of EA's.

**Summary**

Increasing the number of local optima in Multi-Start local search adversely impacted runtime due to amplified exploration costs, convergence challenges, larger search space sizes, algorithm sensitivity, and potential variability in solution quality. However, fitness values among the three approaches showed no significant difference regardless of local optima count. Adjusting the number of generations might accentuate differences. The convergence rate analysis revealed unexpected behavior with 2-way tournament selection showing lower solution arrivals in the first problem (fewer local optima), highlighting the stochastic nature of Evolutionary Algorithms (EAs). 